In [1]:
import numpy as np
import lightgbm as lgb

from scipy.special import softmax
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

from structure.Dataset import Dataset
from parser.lightgbm import parse_lightgbm
from extract import get_lgb_trees

/Users/tomek/.virtualenvs/masters/lib/python3.6/site-packages/lightgbm/__init__.py:47: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [29]:
iris = load_iris()
X_train, X_val, y_train, y_val = train_test_split(iris.data, iris.target, test_size=0.2)
iris_data = Dataset(X_train, y_train, iris.feature_names, iris.target_names)

n = 5
clf = lgb.LGBMClassifier(n_estimators=n, objective='softmax')
clf.fit(X_train, y_train)

trees = parse_lightgbm(clf, iris_data)

In [30]:
trees_preds = np.array([tree.predict(X_val) for tree in trees])
print(f'Shape = {trees_preds.shape}')

Shape = (15, 30)


## Jak LightGBM działa

* Softmax na wartościach `leaf_value` jeśli `objective='multiclass'`
* Suma leaf_value: Podzielone w taki sposób dla klas A, B oraz C: [A, B, C, A, B, C, ...]

In [36]:
clf_preds = clf.predict(X_val)

which = 0
preds = trees_preds[:, which]
y_true = y_val[which]
y_clf = clf_preds[which]

preds = preds.reshape((n, len(iris.target_names)))
probs = softmax(np.sum(preds, axis=0))

In [37]:
print(f'Is {np.argmax(probs)}, clf says {y_clf}, should be {y_true}')

Is 1, clf says 1, should be 1


In [39]:
clf.predict_proba([X_val[which]]), probs

(array([[0.23560392, 0.52519782, 0.23919826]]),
 array([0.23560392, 0.52519782, 0.23919826]))